<a href="https://colab.research.google.com/github/Ashnaaaaa/PROMPT-LOGIC/blob/main/PROMPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#temperature
#what is python
def temptest(temp):
  if temp<0.5:
    return"python is oop"
  else:
   return"python is high level language interpreter"
print(temptest(0.2))
print(temptest(0.8))


python is oop
python is high level language interpreter


In [ ]:
!pip install langchain

In [ ]:
#email classification using prompt Engg
!pip install transformers
from transformers import pipeline
from sklearn.metrics import classification_report
import pandas as pd
#dataset
data={
    "email": ["Hi Aszz,Google is hiring for the role of Software Engineer!Congrats, your profile matches this opportunity!Apply Now .Team Unstop (formerly Dare2Compete)","Welcome back to your account.If you suspect you were locked out of your account because of changes made by someone else, you should review & protect your account","We're Unleashing Door-Opening Deals 😱","Your shipment (AWB 149) is delivered","Upcoming test Reminder: Dear candidate,This is a reminder that yourtest is scheduled to begin in 59 minutes.The test duration is 60 minutes. ","Don't miss conversations in AWS Certification & Training"],
    "label":["spam","transactional","security","primary","social","promotions"]
}
df=pd.DataFrame(data)
classifier=pipeline("zero-shot-classification",model="facebook/bart-large-mnli")
candidate_labels=["spam","transactional","promotions","social","primary","security"]
predicted_labels=[]
for text in df['email']:
  result=classifier(text,candidate_labels)
  predicted_labels.append(result['labels'][0])
df['predicted']=predicted_labels
print(df)
print("\nClassification report:")
print(classification_report(df['label'],df['predicted']))


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


                                               email          label  \
0  Hi Aszz,Google is hiring for the role of Softw...           spam   
1  Welcome back to your account.If you suspect yo...  transactional   
2              We're Unleashing Door-Opening Deals 😱       security   
3               Your shipment (AWB 149) is delivered        primary   
4  Upcoming test Reminder: Dear candidate,This is...         social   
5  Don't miss conversations in AWS Certification ...     promotions   

       predicted  
0        primary  
1       security  
2     promotions  
3  transactional  
4        primary  
5         social  

Classification report:
               precision    recall  f1-score   support

      primary       0.00      0.00      0.00       1.0
   promotions       0.00      0.00      0.00       1.0
     security       0.00      0.00      0.00       1.0
       social       0.00      0.00      0.00       1.0
         spam       0.00      0.00      0.00       1.0
transactional 

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [1]:
import gradio as gr
from transformers import pipeline

# load the zero-shot classification pipeline
# It's better practice to load heavy models outside the function definition
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
# define email classification categories
categories = ["spam", "social", "work", "personal", "finance", "health", "Transactional", "updates", "reminders"]
# classification function using prompt logic
def classify_email(email_text):
    # Ensure multi_label is set to True for potentially multiple relevant categories
    result = classifier(email_text, candidate_labels=categories, multi_label=True)
    # The 'result' dictionary from the pipeline already has 'labels' and 'scores' keys.
    top_result = list(zip(result['labels'], result['scores']))
    # Use a correct lambda function to sort by the second element (score) in the tuple
    sorted_result = sorted(top_result, key=lambda item: item[1], reverse=True)
    top_5 = sorted_result[:5]
    output = "**Top Predictions**\n\n"
    for label, score in top_5:
        # Format the score to two decimal places
        output += f"{label}: {score:.2f}\n"
    return output
# Gradio UI design
with gr.Blocks(theme=gr.themes.Base(), css="""
    body { background: linear-gradient(to right, #e0eafe, #cfde3); }
    /* This targets a CSS class name used internally by Gradio components */
    .gr-box { background-color: #ffffff; border-radius: 12px; padding: 20px; }
    .gr-button { background-color: #008cba !important; color: white; }
""") as app:
    gr.Markdown(
        "<h1 style='text-align:center; color:#333;'>Email classifier using Prompt Logic</h1>"
        "<p style='text-align:center;'>Zero shot classification using Transformers</p>"
    )
    with gr.Row():
        with gr.Column(scale=1):
            email_input = gr.Textbox(label="Enter Email text", lines=10, placeholder="Paste your email here")
        with gr.Column(scale=1):
            result_output = gr.Markdown()

    with gr.Row():
        classify_btn = gr.Button("Classify Email") # Capitalized for better UI
        clear_btn = gr.Button("Clear")

        classify_btn.click(fn=classify_email, inputs=email_input, outputs=result_output)
        # to clear both the input textbox and the output markdown.
        clear_btn.click(fn=lambda: ("", ""), inputs=[], outputs=[email_input, result_output])

# launch the app
app.launch()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d0113a3fb6b6592506.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
